In [3]:
import subprocess
import glob
import os
from scipy.stats import mode


In [4]:
import numpy as np
import os


def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
                
                


def get_neighbours(surfname):
    """Get neighbours from obj file"""
    Polys=[]
    k=0
    with open(surfname,'r') as fp:
        for i, line in enumerate(fp):
            if i==0:
    #Number of vertices
                n_vert=int(line.split()[6])
            elif i==2*n_vert+3:
                n_poly=int(line)
            elif i>2*n_vert+5:
                if not line.strip():
                    k=1
                elif k==1:
                    Polys.extend(line.split())
    Polys=list(map(int, Polys))
    tris=list(chunks(Polys,3))
    neighbours=[[] for i in range(n_vert)]
    for tri in tris:
        neighbours[tri[0]].extend([tri[1],tri[2]])
        neighbours[tri[2]].extend([tri[0],tri[1]])
        neighbours[tri[1]].extend([tri[2],tri[0]])
#Get unique neighbours
    for k in range(len(neighbours)):      
        neighbours[k]=f7(neighbours[k])
    return np.array(neighbours);

def f7(seq):
    #returns uniques but in order to retain neighbour triangle relationship
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))];

def compute_islands(area_txtfile,neighbours):
    """calculates islands with same label value"""
    islands=np.zeros_like(area_txtfile).astype(int)
        #array for quick indexing
    indices=np.arange(len(area_txtfile))
    #k is island counter
    k=0
    #while some vertices haven't been assigned
    while np.sum(islands==0)>0:
        k+=1
        #start with lowest unassigned vertex
        cluster=[np.min(np.where(islands==0)[0])]
        #set vertex to island value
        islands[cluster]=k
        #get label value (i.e. inside or outside label, 1/0)
        v_seed_label=area_txtfile[cluster[0]]

        old_cluster=0
        #while size of island k increases
        while np.sum(islands==k)>np.sum(old_cluster):
            #calculate new vertices
            added_vertices=islands==k-old_cluster
            #store for next comparison
            old_cluster=islands==k
            # for the new vertices
            for v in indices[added_vertices]:
                #get the neighbours
                neighbourhood=np.array(neighbours[v])
                #of the neighbours, which are in the same label and not yet part of the island.
                #set these to the island value
                islands[neighbourhood[np.logical_and(area_txtfile[neighbourhood]==v_seed_label,
                                                     islands[neighbourhood]!=k)]]=k
    return islands


# def tidy_holes(area_txtfile, neighbours,threshold_area=50, iterations=2):
#     """fills small holes in binary surface annotation file.
#     threshold_area is maximum area to be switched.
#     iterations is the number of times looped through in case of nested holes"""
#     #create empty surf file to be filled with island indices
#     for i in range(iterations):
#         islands=np.zeros_like(area_txtfile).astype(int)
#         #array for quick indexing
#         indices=np.arange(len(area_txtfile))
#         #k is island counter
#         k=0
#         #while some vertices haven't been assigned
#         while np.sum(islands==0)>0:
#             k+=1
#             #start with lowest unassigned vertex
#             cluster=[np.min(np.where(islands==0)[0])]
#             #set vertex to island value
#             islands[cluster]=k
#             #get label value (i.e. inside or outside label, 1/0)
#             v_seed_label=area_txtfile[cluster[0]]

#             old_cluster=0
#             #while size of island k increases
#             while np.sum(islands==k)>np.sum(old_cluster):
#                 #calculate new vertices
#                 added_vertices=islands==k-old_cluster
#                 #store for next comparison
#                 old_cluster=islands==k
#                 # for the new vertices
#                 for v in indices[added_vertices]:
#                     #get the neighbours
#                     neighbourhood=np.array(neighbours[v])
#                     #of the neighbours, which are in the same label and not yet part of the island.
#                     #set these to the island value
#                     islands[neighbourhood[np.logical_and(area_txtfile[neighbourhood]==v_seed_label,
#                                                          islands[neighbourhood]!=k)]]=k

#         #then fill the holes
#         new_area=np.copy(area_txtfile).astype(int)
#         island_index,counts=np.unique(islands,return_counts=True)
#         ordered=np.argsort(counts)
#         for ordered_index in ordered:
#             if counts[ordered_index]<threshold_area:
#                 island_i=island_index[ordered_index]
#                 new_area[islands==island_i]=(area_txtfile[islands==island_i][0]-1)*-1
#         area_txtfile=new_area
#     return new_area

def tidy_holes_binary(area_txtfile, neighbours,threshold_area=50, iterations=2):
    """fills small holes in binary surface annotation file.
    threshold_area is maximum area to be switched.
    iterations is the number of times looped through in case of nested holes"""
    #create empty surf file to be filled with island indices
    for i in range(iterations):
        islands=compute_islands(area_txtfile,neighbours)
        #then fill the holes
        new_area=np.copy(area_txtfile).astype(int)
        island_index,counts=np.unique(islands,return_counts=True)
        ordered=np.argsort(counts)
        for ordered_index in ordered:
            if counts[ordered_index]<threshold_area:
                island_i=island_index[ordered_index]
                new_area[islands==island_i]=(area_txtfile[islands==island_i][0]-1)*-1
        area_txtfile=new_area
    return new_area

def tidy_combined_atlas(combined_areas,neighbours,threshold=100):
    """fill holes in combined_atlas, including overlapping areas
    fills values are the most frequent values on the border of the island"""
    all_areas_islands = compute_islands(combined_areas,neighbours)
    island_index,counts=np.unique(all_areas_islands,return_counts=True)
    ordered=np.argsort(counts)
    overlap_values=np.unique(combined_areas[overlaps])
    vertex_indices=np.arange(len(combined_areas))
    new_combined_areas=combined_areas.copy()
    for ordered_index in ordered:
        island_of_interest=all_areas_islands==island_index[ordered_index]
        island_value=combined_areas[island_of_interest][0]
        #only replace islands of 0s or overlaps
        if np.logical_and(counts[ordered_index]<threshold, island_value==0 or (overlaps[island_of_interest]).any()):
        
            neighbours_island=neighbours[island_of_interest]
            long=[]
            for n in neighbours_island:
                long.extend(n)
            unique_neighbours=np.setdiff1d(np.unique(long),vertex_indices[island_of_interest])
            new_combined_areas[island_of_interest]=mode(new_combined_areas[unique_neighbours])[0][0]
    return new_combined_areas

In [7]:
outdir='../surface_data/area_labels'
os.makedirs(outdir,exist_ok=True)
area_blocks=glob.glob('../volume_data/area_blocks/*.mnc')
area_names=[]
for ab in area_blocks:
    ab=ab.replace('_block.mnc','')
    ab=ab.replace('../volume_data/area_blocks/','')
    area_names.append(ab)

area_names=sorted(area_names)

In [257]:
hemis=['right','left']
for k,area in enumerate(area_names):
    print(area)
    for hemi in hemis:
        mid_surface_name='../surface_data/obj_surfaces/mid_{}.obj'.format(hemi)

        subprocess.call('volume_object_evaluate -nearest_neighbour {} {} {}'.format(area_blocks[k],
                                                             mid_surface_name,
                                                             os.path.join(outdir,'{}_{}.txt'.format(area,hemi))),shell=True)
       

  
        
        

auditory_Te3
ifs_ifj2
pIPS_hIP4
area6d_6d2
auditory_Te11
hippocampus_EC
pIPS_hIP7
sts_Te4
pIPS_hIP8
pIPS_hIP6
sma_sma
pIPS_hIP5
sma_presma
ifs_ifs3
v2
sts_Te5
auditory_Te10
area6d_6d3
pIPS_hOc6
ifs_ifs1
ifs_ifs2
ifs_ifj1
v1
auditory_Te12
ifs_ifs4
pIPS_hPO1
area6d_6d1


In [259]:
#tidy up individual areas
neighbours=get_neighbours(mid_surface_name)
for k,area in enumerate(area_names):
    print(area)
    
    for hemi in hemis:
       #fill holes
        area_txtfile=np.loadtxt(os.path.join(outdir,'{}_{}.txt'.format(area,hemi))).astype(bool)
        
        tidied=tidy_holes_binary(area_txtfile,neighbours, threshold_area=200,iterations=2)
        np.savetxt(os.path.join(outdir,'{}_{}_tidied.txt'.format(area,hemi)),tidied,fmt='%i')
      

auditory_Te3
ifs_ifj2
pIPS_hIP4
area6d_6d2
auditory_Te11
hippocampus_EC
pIPS_hIP7
sts_Te4
pIPS_hIP8
pIPS_hIP6
sma_sma
pIPS_hIP5
sma_presma
ifs_ifs3
v2
sts_Te5
auditory_Te10
area6d_6d3
pIPS_hOc6
ifs_ifs1
ifs_ifs2
ifs_ifj1
v1
auditory_Te12
ifs_ifs4
pIPS_hPO1
area6d_6d1


In [20]:
#combine areas into single parcellation
hemis=['left','right']
for hemi in hemis:
    area_labels=['null']
    mid_surface_name='../surface_data/obj_surfaces/mid_{}.obj'.format(hemi)
    neighbours=get_neighbours(mid_surface_name)

    combined_areas = np.zeros(len(neighbours))
    overlaps = np.zeros(len(tidied)).astype(bool)
    for k,area in enumerate(area_names):
        print(area)
        area_txtfile=np.loadtxt(os.path.join(outdir,'{}_{}_tidied.txt'.format(area,hemi)))
        overlaps+= np.logical_and(area_txtfile,combined_areas)
        combined_areas+= area_txtfile*(k+1)
        area_labels.append(area)
    tidied_combined=tidy_combined_atlas(combined_areas, neighbours,threshold=200)
    
    np.savetxt(os.path.join(outdir,'combined_areas_{}_tidied.txt'.format(hemi)),tidied_combined, fmt='%i')
    np.savetxt(os.path.join(outdir,'area_labels_{}.txt'.format(hemi)),area_labels,fmt='%s')

area6d_6d1
area6d_6d2
area6d_6d3
auditory_Te10
auditory_Te11
auditory_Te12
auditory_Te3
hippocampus_EC
ifs_ifj1
ifs_ifj2
ifs_ifs1
ifs_ifs2
ifs_ifs3
ifs_ifs4
pIPS_hIP4
pIPS_hIP5
pIPS_hIP6
pIPS_hIP7
pIPS_hIP8
pIPS_hOc6
pIPS_hPO1
sma_presma
sma_sma
sts_Te4
sts_Te5
v1
v2
area6d_6d1
area6d_6d2
area6d_6d3
auditory_Te10
auditory_Te11
auditory_Te12
auditory_Te3
hippocampus_EC
ifs_ifj1
ifs_ifj2
ifs_ifs1
ifs_ifs2
ifs_ifs3
ifs_ifs4
pIPS_hIP4
pIPS_hIP5
pIPS_hIP6
pIPS_hIP7
pIPS_hIP8
pIPS_hOc6
pIPS_hPO1
sma_presma
sma_sma
sts_Te4
sts_Te5
v1
v2
